Arquivo: train_fretboard_yolo.py

Objetivo:
Treinar um modelo YOLO para detectar o braço da guitarra (fretboard)
em frames de vídeo, em múltiplos ângulos e perspectivas.

Pré-requisitos:
- Dataset anotado no formato YOLO
- fretboard.yaml configurado corretamente

In [11]:
from ultralytics import YOLO
import os
import cv2

In [12]:
DATA_YAML = "fretboard.yaml"
OUTPUT_DIR = "models"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [13]:
from ultralytics import settings
print(settings)

JSONDict("C:\Users\marce\AppData\Roaming\Ultralytics\settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "./yolo_dataset",
  "weights_dir": "./yolo_dataset/runs",
  "runs_dir": "./yolo_dataset/runs",
  "uuid": "35eb81672b94382f3beb811194690c6620ca3b7c24d7b96710826297a3c897fc",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": false,
  "wandb": false,
  "vscode_msg": true,
  "openvino_msg": true
}


In [ ]:
from pathlib import Path

print(Path("fretboard.yaml").resolve())
print(Path("fretboard.yaml").exists())

In [ ]:
model = YOLO("yolo11n.pt")

In [ ]:
model.train(
    data=DATA_YAML,      # <-- dataset + labels
    epochs=100,          # ajuste conforme dataset
    patience=10,       # para evitar overfitting
    batch=16,
    imgsz=640,
    workers=8,
    pretrained=True,
    resume=False,
    single_cls=False,
    project=OUTPUT_DIR,
    name="fretboard_yolo",
    exist_ok=True,
    box=7.5,  # More recall, better IoU, less precission, 
    cls=0.5,  # Bbox class better
    dfl=1.5,  # Distribution Focal Loss. Better bbox boundaries
    val=True,
    # Argumentos
    degrees=0.3,
    hsv_s=0.3,
    hsv_v=0.3,
    scale=0.5,
    fliplr=0.5,
)

In [ ]:
model.save(os.path.join(OUTPUT_DIR, "fretboard_yolo.pt"))

In [ ]:
model = YOLO('runs/detect/train/weights/best.pt')

In [ ]:
results = model.val(#project='runs/detect',
                    imgsz=640,
                    batch=16,
                    conf=0.001,
                    iou=0.7,  # Non-Maximum Supression (NMS)
                    save_json=False,  # Save to JSON {image_id, cls, bbox, conf} of each image in dataset
                    save_hybrid=False,  # Bounding box labels + inference on the output image (Ultralytics 8.0.178 = results are wrong when this argmunent is True)
                    split='test')  # train, val or test

In [ ]:
results

In [ ]:
results.box

In [ ]:
results.box.map50